# End to End Example of Using the `aisquared` Python SDK to Create a Dashboard Integration Using a `ReverseMLWorkflow`

In this tutorial, we will go over the creation of a `.air` file that uses a `ReverseMLWorkflow` to pull data from an Amazon S3 bucket based on the customer Dynamics 365 page the user is viewing and present a dashboard containing tables using that data.

# Importing Packages

Because the data to be used for the Reverse ML Workflow is already stored in S3, we only need to import the `config` subpackage from the `aisquared` package to complete this tutorial.

In [1]:
# import the aisquared package
from aisquared import config

2023-02-02 08:41:42.788342: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 08:41:42.911567: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-02 08:41:42.911599: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-02 08:41:42.934860: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-02 08:41:43.453540: W tensorflow/stream_executor/platform/de

# Defining the Workflow

Now that we have imported the required package, let's dive head first into completing this workflow.  We are going to need to perform the following steps:

- Create a harvester which looks for customer names in the webpage
- Connect the webpage and the text we have harvested to the S3 bucket for query using the `ReverseMLWorkflow`
- Render results in a set of steps to create the dashboard using the data related to the customer

In [ ]:
# Create the harvester
harvester = config.harvesting.TextHarvester(
    how = 'regex',
    regex = 'Alice Alison|Bob Bobson|Charles Charleston|David Davidson|Eric Ercison|Frederick Frederickson|Georg Georges|Hilda Henrick|Igor Igloo|John Johnson',
    limit = 1
)

# No preprocesser is required for this dashboard
preprocesser = None

# The analytic in this workflow is a ReverseMLWorkflow
analytic = config.analytic.ReverseMLWorkflow(
    column = 'Lead_Name',
    period = 1,
    filenames = 'SOS_Sample_for_Dashboard_edit9.csv',
    input_type = 'text',
    bucket = 'ai2-demo-bucket',
    filter_type = 'group',
    filter_by_columns = []
)

# No postprocesser is required for this dashboard
postprocesser = None

# Rendering steps for the dashboard
rendering_steps = [

    # Create the global container that will encompass the entire dashboard
    config.rendering.ContainerRendering(
        label = 'Dashboard Container',
        id = 'ai2-sos-dashboard',
        query_selector = "[data-id='tabpanel-SUMMARY_TAB']"
    ),
    
    # Create the top container that will be embedded in the global container
    config.rendering.ContainerRendering(
        label = 'Top Half',
        id = 'ai2-sos-dashboard-top',
        query_selector = '#ai2-sos-dashboard',
        width = '100',
        height = 'auto'
    ),
    
    # Create the top left container embedded in the top container
    config.rendering.ContainerRendering(
        label = 'Top Left Data Container',
        id = 'ai2-sos-dashboard-firm-score',
        query_selector = '#ai2-sos-dashboard-top'
    ),
    
    # Create an HTML heading in the top container
    config.rendering.HTMLTagRendering(
        label = 'Lead Name Label',
        id = 'sos-contact-name-label',
        container_id = 'ai2-sos-dashboard-firm-score',
        html_content = '<h1 id="sos-contact-name-label"></h1>',
        extra_content_tag = 'strong',
        injection_action = 'prepend',
        prediction_name_key = 'name',
        prediction_value_key = 'value',
        prediction_name_value = 'Lead_Name'
    ),
    
    # Render a table of pertinent information retrieved from the ReverseML Workflow
    config.rendering.TableRendering(
        label = 'Firm Market Table',
        id = 'lead-firm-data-table',
        table_name = '',
        container_id = 'ai2-sos-dashboard-firm-score',
        prediction_name_key = 'name',
        prediction_value_key = 'value',
        prediction_name_values = ['FirmName', 'AUM', 'NCF', 'Marketshare']
    ),
    
    # Render another table
    config.rendering.TableRendering(
        label = 'Engagement Trends Table',
        id = 'ai2-sos-engagment-table',
        table_name = 'Engagement Trends YTD',
        container_id = 'ai2-sos-dashboard-firm-score',
        prediction_name_key = 'name',
        prediction_value_key = 'value',
        prediction_name_values = ['Web Visits', 'Marketing Emails', 'Outbound Calls', 'Inbound Calls']
    ),
    
    # Create another heading
    config.rendering.HTMLTagRendering(
        label = 'Top 3 Core Label',
        id = 'top-3-core-product-label',
        content = 'Top 3 Core Product Rx',
        container_id = 'ai2-sos-dashboard-firm-score',
        html_content = '<h1 id="top-3-core-product-label">Top 3 Core Product Rx</h1>',
        extra_content_tag = 'strong',
        injection_action = 'prepend',
        prediction_name_key = '',
        prediction_value_key = '',
        prediction_name_value = ''
    ),
    
    # Create an HTML tag for core recommendations
    config.rendering.HTMLTagRendering(
        label = 'Top Core Values',
        id = 'top-3-core-values',
        content = '',
        container_id = 'ai2-sos-dashboard-firm-score',
        html_content = '<p id="top-3-core-values"></p>',
        extra_content_tag = 'strong',
        injection_action = 'prepend',
        prediction_name_key = 'name',
        prediction_value_key = 'value',
        prediction_name_value = 'Core Recommendations'
    ),
    
    # Create an HTML tag for top 3 non-core product recommendations
    config.rendering.HTMLTagRendering(
        label = 'Top 3 Non-core Label',
        id = 'top-3-non-core-product',
        content = 'Top 3 non-core Product Rx',
        container_id = 'ai2-sos-dashboard-firm-score',
        html_content = '<h1 id="top-3-non-core-product">Top 3 Non-core Product Rx</h1>',
        extra_content_tag = 'h1',
        injection_action = 'prepend',
        prediction_name_key = '',
        prediction_value_key = '',
        prediction_name_value = ''
    ),
    
    # Create and HTML tag for top 3 non-core values
    config.rendering.HTMLTagRendering(
        label = 'Top Non-Core Values',
        id = 'top-3-non-core-values',
        content = '',
        container_id = 'ai2-sos-dashboard-firm-score',
        html_content = '<p id="top-3-non-core-values"></p>',
        extra_content_tag = 'strong',
        injection_action = 'prepend',
        prediction_name_key = 'name',
        prediction_value_key = 'value',
        prediction_name_value = 'Non-Core Recommendations'
    ),

    # Create the Doughnut Chart
    config.rendering.PieChartRendering(
        label = 'Sales Opp Chart',
        id = 'ai2-sos-score-chart',
        chart_name = 'Sales Opp Score',
        legend_icon = 'circle',
        container_id = 'ai2-sos-dashboard-top',
        labels = None,
        labels_key = 'Lead_Name',
        consolidate_rows = True,
        prediction_name_key = 'name',
        prediction_value_key = 'value',
        prediction_name_value = 'Score',
        display_legend = True
    )
]

# Compile Into a `.air` File

Now that we have all of the steps, all we have to do is put them together and compile them into a `.air` file to be uploaded into the platform!

In [ ]:
# Compile all of the configuration information into a ModelConfiguration class, to
# be compiled into a .air file
model_config = config.ModelConfiguration(
    'Python SDK SOS Dashboard Dynamics 365',
    harvesting_steps=harvester,
    preprocessing_steps=preprocesser,
    analytic=analytic,
    postprocessing_steps=postprocesser,
    rendering_steps=rendering_steps,
    description='Version of the SOS Dashboard .air file created using the Python SDK'
)

# Compile the model
model_config.compile()


# Upload the file to the platform

We are done!  Now, you should have a file called `Python SDK SOS Dashboard` in this directory.  Go ahead and upload the file to the platform to start seeing results!